# Задача мэтчинга товаров

**Цели исследования:**  
  
Для каждого товара магазина найти один или несколько объектов из ассортимента магазина-конкурента, которые близки к нему по некоторой заданной метрике. 

**Задача:**
  
Разработать модель для метчинга товаров в соответствии с требуемой метрикой.
- разработать алгоритм, который для всех товаров магазина предложит несколько вариантов наиболее похожих товаров из ассортимента магазина-конкурента;
- оценить качество алгоритма по метрике accuracy@5

  
**План работы:**
  
- загрузить и изучить представленные данные;
- провести необходимую предобработку данных;
- провести исследовательский анализ данных;
- провести корреляционный анализ признаков, сделать выводы о мультиколлинеарности и при необходимости устранить её.
- выполнить подготовку признаков в пайплайне;
- выбрать лучшую модель и проверить её качество;
- провести анализ важности признаков, сделать выводы об их значимости;
- сформировать выводы и рекомендации по каждому шагу исследования;
- сформировать общий вывод и рекомендации.

**Какими данными располагаем:** 
  
- `base.csv` - анонимизированный набор товаров. Каждый товар представлен как уникальный id (0-base, 1-base, 2-base) и вектор признаков размерностью 72.
- `train.csv` - обучающий датасет. Каждая строчка - один товар, для которого известен уникальный id (0-query, 1-query, …) , вектор признаков И id товара из base.csv, который максимально похож на него (по мнению экспертов).
- `validation.csv` - датасет с товарами (уникальный id и вектор признаков), для которых надо найти наиболее близкие товары из base.csv
- `validation_answer.csv` - правильные ответы к предыдущему файлу.

**Для начала импортируем библиотеки:**

In [42]:
%%time
%%capture

# Стандартные библиотеки
import os
import re
import sys
import time
import warnings
from datetime import datetime
from math import ceil

# Апдейт и установка необходимых пакетов
!"{sys.executable}" -m pip install -U numba
!"{sys.executable}" -m pip install numpy==1.26.4
!"{sys.executable}" -m pip install scipy==1.13.1
!"{sys.executable}" -m pip install pandas==1.4.4
!"{sys.executable}" -m pip install --upgrade scikit-learn
!"{sys.executable}" -m pip install --upgrade matplotlib
!"{sys.executable}" -m pip install --upgrade seaborn
!"{sys.executable}" -m pip install --upgrade jinja2==3.1.4
!"{sys.executable}" -m pip install catboost
!"{sys.executable}" -m pip install missingno
!"{sys.executable}" -m pip install phik
!"{sys.executable}" -m pip install shap
!"{sys.executable}" -m pip install tqdm 

# Сторонние библиотеки
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import phik
import seaborn as sns
import shap
from IPython.display import display, HTML
from matplotlib.axes._axes import _log as matplotlib_axes_logger
from matplotlib.ticker import MultipleLocator
import missingno as msno
from pandas.plotting import register_matplotlib_converters
from scipy import stats as st

import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer, VectorAssembler, RobustScaler, OneHotEncoder 
from pyspark.ml.stat import Correlation
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.regression import LinearRegression

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

# # Библиотеки scikit-learn
# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.compose import ColumnTransformer
# from sklearn.dummy import DummyRegressor
# from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
# from sklearn.experimental import enable_halving_search_cv
# from sklearn.impute import SimpleImputer
# from sklearn.inspection import permutation_importance
# from sklearn.linear_model import LinearRegression, Lasso, Ridge
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
# (
#     GridSearchCV, RandomizedSearchCV, HalvingGridSearchCV, train_test_split
# )
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import (
#     LabelEncoder, MinMaxScaler, OneHotEncoder, OrdinalEncoder, RobustScaler, StandardScaler
# )
# from sklearn.tree import DecisionTreeRegressor
from sklearn.utils import shuffle

# FAISS
import faiss

# Дополнительные библиотеки
from tqdm import tqdm

# Дополнительные настройки
matplotlib_axes_logger.setLevel('ERROR')
warnings.filterwarnings("ignore")
warnings.warn("ignore")
register_matplotlib_converters()

# Зафиксированные параметры визуализации
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100) 
sns.set(rc={'figure.figsize': (20, 10)})
mpl.rcParams.update({'font.size': 11})
sns.set_style("whitegrid")

CPU times: user 51.9 ms, sys: 70.1 ms, total: 122 ms
Wall time: 14.7 s


**Функции, используемые в рамках исследования**

In [2]:
def load_pd_df(file_name, parse_dates=None, sep=None, dec=',', index_col=0):
    """
    Ищет файл в сети и локально, загружает его и возвращает как pandas DataFrame.
    Также умеет парсить дату по столбцам.

    :param file_name: имя файла для загрузки
    :param sep: разделитель колонок в файле (например, ',' для CSV)
    :param dec: символ десятичного разделителя (по умолчанию ',')
    :param parse_dates: список столбцов, которые нужно разобрать как даты
    :return: загруженный DataFrame или None, если файл не найден
    """
    if parse_dates is None:
        parse_dates = []
    file_path_net = f'/datasets/{file_name}'
    file_path_local = file_name

    try:
        if os.path.exists(file_path_net):
            file_path = file_path_net
        elif os.path.exists(file_path_local):
            file_path = file_path_local
        else:
            print(f'[ ❌ ] {file_name} не найден нигде')
            return None
        
        df = pd.read_csv(file_path, parse_dates=parse_dates, sep=sep, decimal=dec, index_col=index_col)
        location = "сети" if file_path == file_path_net else "локального хранилища"
        print(f'[ 👍 ] {file_name} успешно загружен из {location}')
        return df
    except Exception as e:
        print(f'Произошла ошибка при загрузке: {e}')
        return None

In [3]:
def load_spark_df(file_name, sep=None, header=True, inferSchema=True):
    """
    Ищет файл в сети и локально, загружает его и возвращает как Spark DataFrame.
    Также умеет парсить дату по столбцам.

    :param df_name: имя датафрейма
    :param file_name: имя файла для загрузки
    :param sep: разделитель колонок в файле
    :param header: заголовок (по умолчанию True)
    :param inferSchema: автоматическая схема данных (по умолчанию True)
    :return: загруженный Spark DataFrame или None, если файл не найден
    """
    file_path_net = f'/datasets/{file_name}'
    file_path_local = file_name
    
    try:
        if os.path.exists(file_path_net):
            file_path = file_path_net
        elif os.path.exists(file_path_local):
            file_path = file_path_local
        else:
            print(f'[ ❌ ]{file_name} не найден нигде')
            return None
        
        df = spark.read.csv(file_path, sep=sep, header=header, inferSchema=inferSchema)
        location = "сети" if file_path == file_path else "локального хранилища"
        print(f'[ 👍 ] {file_name} успешно загружен из {location}')
        return df
    except Exception as e:
        print(f'Произошла ошибка при загрузке: {e}')
        return None

In [8]:
def look_on(df):
    display(df.head())
    df.info()

In [33]:
def spark_info(df):
    # Создаем выражения агрегирования для подсчета ненулевых значений в каждом столбце
    agg_exprs = [F.count(F.col(c)).alias(c) for c in df.columns]
    
    # Применяем агрегирование ко всему DataFrame сразу
    non_null_counts = df.agg(*agg_exprs).collect()[0].asDict()
    
    # Получаем типы данных для каждого столбца
    dtypes_dict = dict(df.dtypes)
    
    # Формируем список словарей для последующего создания pandas DataFrame
    rows = []
    for column, dtype in dtypes_dict.items():
        row = {
            'Column Name': column,
            'Non-Null Count': non_null_counts[column],
            'Data Type': dtype
        }
        rows.append(row)
    
    # Возвращаем pandas DataFrame с информацией о столбцах
    return pd.DataFrame(rows)

In [32]:
def print_nulls(df):
    # Создаем выражения для агрегирования: подсчет количества null значений в каждом столбце
    agg_exprs = [F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]
    
    # Применяем агрегирование ко всему DataFrame сразу
    nulls_df = df.agg(*agg_exprs)
    
    # Собираем результат в локальный объект Python для дальнейшей обработки
    nulls_counts = nulls_df.collect()[0].asDict()
    
    # Флаг для отслеживания наличия null значений
    nulls_found = False
    
    # Перебираем полученные результаты и печатаем информацию о столбцах с null значениями
    for column, count in nulls_counts.items():
        if count > 0:
            print(f"{column}: {count}")
            nulls_found = True
    
    # Если null значения не найдены
    if not nulls_found:
        print("Пропусков не обнаружено")

In [34]:
def double_to_float(df):
    columns = base_spark.columns
    
    # Если формат 'double' тогда меняем на 'float'
    for column in columns:
        if dict(df.dtypes)[column] == 'double':
            df = df.withColumn(column, F.col(column).cast(FloatType()))
    # Печатаем результат        
    print(pd.DataFrame(base_spark.dtypes))        
    return df

In [48]:
def optimize_memory_usage(df, print_size=True):
    """
    Функция для оптимизации использования памяти в DataFrame.

    Параметры:
    :df: - таблица данных.
    :print_size: bool - флаг для вывода результатов оптимизации.

    :return:
    df - DataFrame с оптимизированным использованием памяти.
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    # Типы, которые будем проверять на оптимизацию

    # Размер занимаемой памяти до оптимизации (в Мб)
    before_size = df.memory_usage().sum() / (1024**2)

    for column in df.columns:
        column_type = df[column].dtypes
        if column_type in numerics:
            column_min = df[column].min()
            column_max = df[column].max()
            if str(column_type).startswith('int'):
                if column_min > np.iinfo(np.int8).min and column_max < np.iinfo(np.int8).max:
                    df[column] = df[column].astype(np.int8)
                elif column_min > np.iinfo(np.int16).min and column_max < np.iinfo(np.int16).max:
                    df[column] = df[column].astype(np.int16)
                elif column_min > np.iinfo(np.int32).min and column_max < np.iinfo(np.int32).max:
                    df[column] = df[column].astype(np.int32)
                elif column_min > np.iinfo(np.int64).min and column_max < np.iinfo(np.int64).max:
                    df[column] = df[column].astype(np.int64)
            else:
                if column_min > np.finfo(np.float32).min and column_max < np.finfo(np.float32).max:
                    df[column] = df[column].astype(np.float32)
                else:
                    df[column] = df[column].astype(np.float64)

    # Размер занимаемой памяти после оптимизации (в Мб)
    after_size = df.memory_usage().sum() / (1024**2)

    if print_size:
        print('Размер использования памяти: до {:5.2f} Mb - после {:5.2f} Mb ({:.1f}%)'
              .format(before_size, after_size, 100 * (before_size - after_size) / before_size))

    return df

## Загрузка и обзор данных

Инициализируем Spark-сессию.

In [5]:
%%time
spark = SparkSession.builder \
                    .master("local") \
                    .appName("EDA California Housing") \
                    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/29 20:11:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


CPU times: user 6.54 ms, sys: 7.2 ms, total: 13.7 ms
Wall time: 1.68 s


In [6]:
# Загрузка sample
base_sample = load_pd_df('sample/base.csv')
train_sample = load_pd_df('sample/train.csv')
validation_answer_sample = load_pd_df('sample/validation_answer.csv')
validation_sample = load_pd_df('sample/validation.csv')

# Загрузка основных данных
base_spark = load_spark_df('all/base.csv') # загружаем как Spark DataFrame
train = load_pd_df('all/train.csv')
validation_answer = load_pd_df('all/validation_answer.csv')
validation = load_pd_df('all/validation.csv')

[ 👍 ] sample/base.csv успешно загружен из локального хранилища


24/07/29 20:11:55 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


[ 👍 ] sample/train.csv успешно загружен из локального хранилища
[ 👍 ] sample/validation_answer.csv успешно загружен из локального хранилища
[ 👍 ] sample/validation.csv успешно загружен из локального хранилища


[ 👍 ] all/base.csv успешно загружен из сети
[ 👍 ] all/train.csv успешно загружен из локального хранилища
[ 👍 ] all/validation_answer.csv успешно загружен из локального хранилища
[ 👍 ] all/validation.csv успешно загружен из локального хранилища


Загрузим оба набора данных, где sample - сокращенная версия для отладки модели и тестирования гипотез.
Предобработку и EDA будем проводить на основной базе данных.

Посмотрим на содержимое датафрейма:

**- base:**

In [24]:
print(pd.DataFrame(base_spark.dtypes, columns=['column', 'type']).head(10)) 
base_spark.show(10)

  column    type
0     Id  string
1      0  double
2      1  double
3      2  double
4      3  double
5      4  double
6      5  double
7      6  double
8      7  double
9      8  double
+------+----------+---------+----------+----------+---------+----------+-------------------+----------+----------+----------+-----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+-----------+------------------+-----------+---------+----------+------------------+-----------+---------+----------+---------+----------+----------+----------+-------------------+----------+---------+-----------+----------+----------+----------+----------+-----------+----------+---------+-------------------+---------+----------+----------+----------+----------+-----------+----------+---------+-----------+---------+---------+----------+----------+-----------+-------------------+-----------+----------+-----------+---------+----------+------------------+---------+-------

In [43]:
spark_info(base_spark)

,Column Name,Non-Null Count,Data Type
0,Id,2918139,string
1,0,2918139,float
2,1,2918139,float
3,2,2918139,float
4,3,2918139,float
5,4,2918139,float
6,5,2918139,float
7,6,2918139,float
8,7,2918139,float
9,8,2918139,float


In [44]:
print_nulls(base_spark)

Пропусков не обнаружено


71 признак, 2 918 139 значений формата float, без пропусков.

**- train:**

In [45]:
look_on(train)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,Target
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0-query,-53.882748,17.971436,-42.117104,-183.93668,187.517490,-87.144930,-347.360606,38.307602,109.085560,30.413513,-88.08269,-52.695540,-27.692442,4.872923,198.348000,-30.075249,-3.803569,-135.810610,-161.84137,-78.512180,-94.42894,898.436927,-70.140520,78.420360,108.032776,813.770071,-74.790880,12.610422,-183.821840,149.71584,-27.042316,-133.21217,106.420746,-303.939233,48.73079,58.185707,24.250950,-151.22410,-104.282265,-34.492810,-12.587054,2.622891,-120.969920,149.99164,-31.948470,82.314430,-115.830470,-243.30939,6.622036,-132.197660,68.71256,-38.806824,62.937435,-132.65445,89.189026,87.039780,-121.27988,-6.986934,-206.513820,29.485587,-77.029590,-132.38617,-105.427820,70.107360,-155.80257,-101.965943,65.903790,34.457500,62.642094,134.763600,-415.750254,-25.958572,675816-base
1-query,-87.776370,6.806268,-32.054546,-177.26039,120.803330,-83.810590,-94.572749,-78.433090,124.915900,140.331070,-177.60580,-84.995514,42.810810,-57.256332,96.792534,-19.261467,0.739535,50.619213,-155.26703,-78.659430,-92.76149,353.157741,-34.744545,82.487110,-28.450592,813.770071,-137.529630,26.595627,-136.783450,153.35791,48.810093,-115.92215,87.464220,-222.286354,25.12415,91.887140,-30.636870,-136.59314,-140.500120,-43.449757,-7.226884,8.265747,-117.915470,149.15090,-18.751057,95.315384,-60.093273,-83.82058,37.449867,-23.298859,74.06108,-7.139753,75.862400,-112.04511,82.857730,54.067215,-134.00539,-26.142574,-214.632110,-457.848461,21.459618,-137.41136,-40.812233,4.669178,-151.69771,-1.638704,68.170876,25.096191,89.974976,130.589630,-1035.092211,-51.276833,366656-base
2-query,-49.979565,3.841486,-116.118590,-180.40198,190.128430,-50.837620,26.943937,-30.447489,125.771164,211.607820,-86.34656,-35.666546,16.395317,-80.802850,137.908650,-23.532760,-47.256584,-16.650242,-194.50568,-78.372925,-69.32448,1507.231274,-52.500970,-34.165775,52.958652,813.770071,-18.021725,20.951107,-50.321780,158.76062,0.178065,-183.06967,99.053570,-1018.469545,-51.80112,97.766770,-10.865850,-144.42316,-133.819490,-78.902300,-17.200352,4.467452,-63.970737,154.63953,-30.211614,48.527400,-122.406640,-112.71362,53.461838,-31.117260,107.84151,16.482935,77.934480,-95.61873,91.460075,63.119510,-126.93925,8.066627,-195.677670,-163.120000,-72.830000,-139.22307,-52.031662,78.039764,-169.14620,82.144186,66.008220,18.400496,212.409730,121.931470,-1074.464888,-22.547178,1447819-base
3-query,-47.810562,9.086598,-115.401695,-121.01136,94.652840,-109.255410,-775.150134,79.186520,124.003100,242.650650,-146.51707,-159.469850,-13.844755,-6.113928,118.939255,-44.585907,9.559358,14.435648,-156.90683,-78.789320,-78.73709,1507.231274,19.957405,34.834290,-8.820732,813.770071,-125.606800,17.584084,-58.452904,141.28180,-54.959310,-136.98854,63.880493,-1018.469545,89.22893,65.919960,-24.078644,-152.33410,-91.199380,-28.225390,-4.767386,0.158236,-129.128660,122.95837,-30.800995,123.623400,-37.540867,-72.13980,71.240990,-168.115590,118.23645,-18.065195,37.255720,-137.69104,87.500770,62.437290,-131.26064,35.692660,-86.038830,-379.339090,-153.465770,-131.19829,-61.567047,44.515266,-145.41675,93.990981,64.131350,106.061920,83.178760,118.277725,-1074.464888,-19.902788,1472602-base
4-query,-79.632126,14.442886,-58.903397,-147.05254,57.127068,-16.239529,-321.317964,45.984676,125.941284,103.392670,-107.15302,-8.800034,-50.977800,29.457338,143.389310,5.614824,-45.274760,9.643625,-77.55463,-79.066610,-77.92646,1507.231274,16.612400,116.284290,33.754898,813.770071,-105.765335,6.523008,-19.812988,157.69392,-20.604088,-146.59128,78.849570,-780.449185,87.56077,73.036660,16.891030,-144.65790,-116.122150,-19.353254,-7.709266,-5.394988,-140.252120,193.18497,-53.147078,79.869446,-151.131350,-45.05616,79.796234,46.763016,47.68181,-

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 0-query to 99999-query
Data columns (total 73 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       100000 non-null  float64
 1   1       100000 non-null  float64
 2   2       100000 non-null  float64
 3   3       100000 non-null  float64
 4   4       100000 non-null  float64
 5   5       100000 non-null  float64
 6   6       100000 non-null  float64
 7   7       100000 non-null  float64
 8   8       100000 non-null  float64
 9   9       100000 non-null  float64
 10  10      100000 non-null  float64
 11  11      100000 non-null  float64
 12  12      100000 non-null  float64
 13  13      100000 non-null  float64
 14  14      100000 non-null  float64
 15  15      100000 non-null  float64
 16  16      100000 non-null  float64
 17  17      100000 non-null  float64
 18  18      100000 non-null  float64
 19  19      100000 non-null  float64
 20  20      100000 non-null  float64
 21  21  

100000 значений формата float и целевой признак, пропусков также нет.

**- validation:**

In [46]:
look_on(validation)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100000-query,-57.372734,3.597752,-13.213642,-125.92679,110.74594,-81.279594,-461.003172,139.815720,112.880980,75.215750,-131.89280,-140.968570,-57.987164,-22.868887,150.89552,7.965574,17.622066,-34.868217,-216.13855,-80.908730,-52.579520,263.363136,56.266876,66.924710,21.609911,813.770071,-32.782940,20.794031,-79.779076,156.30708,-42.831330,-71.723335,83.283660,-304.174382,1.609402,55.834587,-29.474255,-139.16277,-126.038350,-62.643830,-5.012346,11.984920,-43.084946,190.124000,-24.996636,76.153900,-245.26157,-143.65648,-4.259628,-46.664196,-27.085403,-34.346962,75.530106,-47.171707,92.697320,60.475630,-127.48687,-39.484753,-124.384575,-307.949760,45.506813,-144.19095,-75.51302,52.830902,-143.43945,59.051935,69.28224,61.927513,111.59253,115.140656,-1099.130485,-117.079360
100001-query,-53.758705,12.790300,-43.268543,-134.41762,114.44991,-90.520130,-759.626065,63.995087,127.117905,53.128998,-153.71725,-63.951330,-52.369495,-33.390945,148.61950,-22.483830,15.164185,-56.202000,-153.61438,-79.831825,-101.055480,1203.537156,81.597130,101.018654,56.783424,92.209628,-126.860340,10.382887,-38.523360,165.38391,-77.840485,-169.538680,103.483240,-915.735701,16.109938,14.669937,-38.707085,-149.53838,-138.792920,-36.076176,-2.781422,2.283144,-142.477890,189.953950,-18.408230,90.517050,-95.53100,-259.63605,52.437836,-30.004599,14.502060,-1.071201,66.842670,-161.279890,94.794174,50.419983,-125.07526,-25.169033,-176.176880,-655.836897,-99.238370,-141.53522,-79.44183,29.185436,-168.60590,-82.872443,70.76560,-65.975950,97.07716,123.391640,-744.442332,-25.009320
100002-query,-64.175095,-3.980927,-7.679249,-170.16093,96.44616,-62.377740,-759.626065,87.477554,131.270110,168.920320,-220.30954,-31.378445,-8.788761,2.285323,133.26611,-41.309080,14.305538,-18.231812,-205.53370,-78.160310,-96.607670,1507.231274,-5.964200,34.937443,-56.086887,813.770071,-13.200474,18.966661,-35.110190,151.36850,-17.490252,-145.884300,15.533379,-655.395514,39.412827,62.554955,9.924992,-143.93462,-123.107796,-37.032475,-13.501337,12.913328,-116.038020,176.276150,-45.909942,103.491360,-90.65699,-162.61570,117.128235,13.079479,69.826890,-6.874451,63.707214,-123.851070,91.610820,59.760067,-129.56618,-12.822194,-154.197650,-407.199067,5.522629,-126.81297,-134.79541,37.368730,-159.66231,-119.232725,67.71044,86.002060,137.63641,141.081630,-294.052271,-70.969604
100003-query,-99.286860,16.123936,9.837166,-148.06044,83.69708,-133.729720,58.576403,-19.046660,115.042404,75.206730,-114.27196,-71.406456,-65.349320,24.377070,50.46730,-14.721335,15.069309,-46.682995,-176.60437,-78.690700,-139.227450,325.547112,3.632292,74.929504,-4.802103,813.770071,-52.982597,15.644381,-54.087467,151.30914,21.088570,-134.507890,65.118960,-529.295053,131.565520,67.642700,-22.884491,-145.90652,-86.917330,-11.863579,-22.188885,0.463720,-212.533750,170.522580,-48.092532,99.712555,-194.69241,-141.52318,60.217050,73.386380,118.567856,58.900810,55.569030,-181.091660,83.340485,66.083240,-114.04887,-57.156870,-56.335075,-318.680065,-15.984783,-128.10133,-77.23611,44.100494,-132.53012,-106.318982,70.88396,23.577892,133.18396,143.252940,-799.363667,-89.392670
100004-query,-79.532920,-0.364173,-16.027431,-170.88495,165.45392,-28.291668,33.931936,34.411217,128.903980,102.086914,-76.21417,-26.393860,34.423640,50.938890,157.68318,-23.786497,-33.175415,-0.592607,-193.31854,-79.651030,-91.889786,1358.481072,44.027733,121.527210,46.183000,433.623103,-82.233200,21.068508,-32.940117,149.26895,0.404718,-97.674530,81.719990,-825.644804,9.397169,49.359340,17.725466,-160.16815,-129.367950,-55.532898,-2.597821,-0.226103,-41.369140,92.090195,-58.626857,73.655440,-10.25737,-175.65678,25.395056,47.874825,51.464676,140.951680,58.751133,-215.487640,91.255

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 100000-query to 199999-query
Data columns (total 72 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       100000 non-null  float64
 1   1       100000 non-null  float64
 2   2       100000 non-null  float64
 3   3       100000 non-null  float64
 4   4       100000 non-null  float64
 5   5       100000 non-null  float64
 6   6       100000 non-null  float64
 7   7       100000 non-null  float64
 8   8       100000 non-null  float64
 9   9       100000 non-null  float64
 10  10      100000 non-null  float64
 11  11      100000 non-null  float64
 12  12      100000 non-null  float64
 13  13      100000 non-null  float64
 14  14      100000 non-null  float64
 15  15      100000 non-null  float64
 16  16      100000 non-null  float64
 17  17      100000 non-null  float64
 18  18      100000 non-null  float64
 19  19      100000 non-null  float64
 20  20      100000 non-null  float64
 21

100 тыс. значений, формат не отличается от предыдущих, пропуски отсутствуют.

**- validation_answer:**

In [47]:
look_on(validation_answer)

,Expected
Id,
100000-query,2676668-base
100001-query,91606-base
100002-query,472256-base
100003-query,3168654-base
100004-query,75484-base


<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 100000-query to 199999-query
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Expected  100000 non-null  object
dtypes: object(1)
memory usage: 1.5+ MB


Здесь только таргет, формат корректный, пропусков нет, что ожидаемо.

### Вывод:

Загруженные данные соответствуют описанию, однако смущает формат double и float64, займемся оптимизацией на следующем шаге.

## Предобработка данных

### Форматы

Формат double, он же float64 достаточно тяжеловесный, трансформируем его для начала в float32. Float16 не поддерживается Spark нативно, поэтому перед обучением модели изменим формат на более подходящий.

In [ ]:
base_spark = double_to_float(base_spark)

In [50]:
train = optimize_memory_usage(train)

Размер использования памяти: до 28.99 Mb - после 28.99 Mb (0.0%)


In [51]:
validation = optimize_memory_usage(validation)

Размер использования памяти: до 55.69 Mb - после 28.23 Mb (49.3%)


Память оптимизировали, теперь посмотрим на распределения.

### Распределения

## EDA

Несмотря на то, что данные обезличены, EDA здесь также будет полезен: все ли столбцы имеют одинаковое распределение значений? Есть ли столбцы, которые для модели были бы мало полезны? Есть ли сильно скоррелированные друг с другом столбцы? Может быть, есть смысл на первом этапе подавать в модель не все фичи, а наиболее информативные? Есть ли пропуски? Явные дубликаты? Если есть - что с ними делать? Есть ли аномалии в распределениях? Следующий важный вопрос - не требуется ли масштабирование данных? Ответить на этот вопрос можно, например, замерив метрику с масштабированием и без масштабирования признаков.

## Целевая метрика

Наша целевая метрика - accuracy@n. Собственно, что это такое. Вспомним, что 

$$
Accuracy = \frac{Correct\ predictions}{All\ predictions}
$$

Представим расчет метрики в цикле, перебирая все предложенные моделью ответы. При этом каждое предсказание содержит в себе не 1 ответ, а сразу n, и если среди предложенных вариантов окажется правильный - числитель и знаменатель увеличиваются на 1. А если нет ни одного - то на 1 увеличивается только знаменатель. В нашей задаче n = 5. Хорошо бы добиться accuracy@5 ≥ 0,7. Кстати, легко заметить, что accuracy@1 - это самая обычная accuracy.

## Create FAISS [index](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes) for small dataset


[Guideline](https://github.com/facebookresearch/faiss/wiki/Guidelines-to-choose-an-index)

Hint: Use numpy [ascontigiousarray](https://numpy.org/doc/stable/reference/generated/numpy.ascontiguousarray.html) - object which is stored in one [unbroken block](https://www.educative.io/answers/what-is-the-numpyascontiguousarray-function-in-python) in memory -  to load vectors in FAISS

In [ ]:
dims = base.shape[1]
n_cells = 20
quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)

In [ ]:
%%time
idx_l2.train(np.ascontiguousarray(base.values).astype('float32'))
idx_l2.add(np.ascontiguousarray(base.values).astype('float32'))

In [ ]:
base_index = {k: v for k, v in enumerate(base.index.to_list())}

## 🔍 Search

In [ ]:
targets = train["Target"]
train.drop("Target", axis=1, inplace=True)

In [ ]:
%%time
candidate_number = 5
r, idx = idx_l2.search(np.ascontiguousarray(train.values).astype('float32'), candidate_number)

## 📈 Accuracy@candidate_number calculation

In [ ]:
acc = 0
for target, el in zip(targets.values.tolist(), idx.tolist()):
    acc += int(target in [base_index[r] for r in el])
print(f'Accuracy @ {candidate_number} = {acc / len(idx):.1%}')

In [ ]:
## ❓❓❓ What's next?

For full dataset it is strongly recommended to test your code on the small batch before loading all dataset to FAISS

You can make your own research:
- change number of cells
- change number of candidates
- change indexes
- add another ML models to improve the FAISS result
- change the accelerator: Hint: Search method on GPU differs a bit from the similar method on CPU
-.....

Remember, that in Colab you have only 12 GB of RAM, so remove variables and objects if necessary

**Good Luck!**

In [ ]:
# FAISS
# Annoy
# Qdrant